# Sample Analysis with Example Data

In [16]:
import utils

ModuleNotFoundError: No module named 'sklearn'

## 1. Data Loading

In [17]:
from utils import read_file

ModuleNotFoundError: No module named 'sklearn'